# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [30]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

In [31]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [32]:
# assign the uk_food database to a variable name


uk_food_db = mongo['uk_food']


In [35]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'petsitly_marketing', 'uk_food']


In [34]:
# review the collections in our new database
print(uk_food_db.list_collection_names())

['establishments']


In [39]:

# review a document in the establishments collection
uk_food_db.establishments.find_one()

{'_id': ObjectId('6432d7a6b0672d1edff974cb'),
 'FHRSID': 632212,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000043474',
 'BusinessName': 'Lenox House',
 'BusinessType': 'Hotel/bed & breakfast/guest house',
 'BusinessTypeID': 7842,
 'AddressLine1': '27 Granville Road',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DS',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2020-03-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.385585', 'latitude': '51.152411'},
 'RightToReply': '',
 'Distance': 4587.440906956158,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount':

In [41]:
# assign the collection to a variable
establishments = uk_food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [42]:
# Create a dictionary for the new restaurant data
Resturant_Dict = { 
                  "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True}

In [43]:
# Insert the new restaurant into the collection
establishments.insert_one(Resturant_Dict)

In [44]:
# Check that the new restaurant was inserted

query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6432eb04e0fe4b9e5b5db761'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [45]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}

results = establishments.find(query, fields)

pprint(results[0])


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6432d7a6b0672d1edff974cc')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [47]:
# Update the new restaurant with the correct BusinessTypeID
uk_food_db.establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set':
        {'BusinessTypeID': 1}
    }
)

In [48]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
fields = {'BusinessName': 1, 'BusinessTypeID': 1}

results = establishments.find(query, fields)
pprint(results[0])

{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('6432eb04e0fe4b9e5b5db761')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [49]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
print('Number of documents: ', establishments.count_documents(query))

Number of documents:  994


In [50]:
# Delete all documents where LocalAuthorityName is "Dover"

query = {'LocalAuthorityName': 'Dover'}
establishments.delete_many(query)


In [ ]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
print('Number of documents: ', establishments.count_documents(query))

In [51]:
# Check that other documents remain with 'find_one'

pprint(uk_food_db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6432d7a6b0672d1edff977b0'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [52]:
# Change the data type from String to Decimal for longitude

establishments.update_many({}, [{'$set': {'geocode.longitude':\
                                          {'$toDecimal': '$geocode.longitude'}}}])

In [53]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [{'$set': {'geocode.latitude':\
                                          {'$toDecimal': '$geocode.latitude'}}}])

In [54]:
# Check that the coordinates are now numbers

document = establishments.find_one()
geocode_dict = document['geocode']

longitude_type = type(geocode_dict['longitude'])
print("The data type of 'longitude' is: ", longitude_type)

latitude_type = type(geocode_dict['latitude'])
print("The data type of 'latitude' is: ", latitude_type)

The data type of 'longitude' is:  <class 'bson.decimal128.Decimal128'>
The data type of 'latitude' is:  <class 'bson.decimal128.Decimal128'>
